In [16]:
import itertools
from collections import Counter
from random import shuffle
import math

relevances = { 'N':0, 'R':1, 'HR':2 }
rankingsOf5 = list(itertools.product(list(relevances.keys()), repeat=5))
pairsOfRankingsOf5 = list(itertools.product(rankingsOf5, rankingsOf5))
shuffle(pairsOfRankingsOf5)
print(len(pairsOfRankingsOf5), "pairs of rankings")

def getContingencies (items, k, relevantDocumentCount):
    retrievedCounter = Counter(items[:k])
    TP = retrievedCounter['R'] + retrievedCounter['HR']
    FP = retrievedCounter['N']
    
    notRetrievedCounter = Counter(items[k:])
    TN = notRetrievedCounter['N']
    FN = relevantDocumentCount - TP
    
    return TP, FP, TN, FN

def getPrecisionAtK (ranking, k):
    TP, FP, TN, FN = getContingencies(ranking, k, relevantDocumentCount)
    precisionAtK = TP / (TP + FP)
#     recallAtK = TP / (TP + FN)
#     F1AtK = 2*precisionAtK*recallAtK
#     if F1AtK > 0.0: 
#         F1AtK /= precisionAtK + recallAtK
#     accuracyAtK = (TP + TN)/(TP + FP + FN + TN)
    return precisionAtK

def getAveragePrecision (ranking, relevantDocumentCount):
    precisionsForAp = []
    for k in range(1, len(ranking)+1):
        precisionAtK = getPrecisionAtK(ranking, k)

        if ranking[k-1] == 'R' or ranking[k-1] == 'HR':
            # save for calculating AP later
            precisionsForAp.append(precisionAtK)
    
    averagePrecision = sum(precisionsForAp)/relevantDocumentCount
    return averagePrecision

def getDiscountedCumulativeGain (ranking):
    dcg = 0.0
    for r in range(1, len(ranking)+1):
        relevanceAtR = 0
        if ranking[r-1] == 'R':
            relevanceAtR = 1
        elif ranking[r-1] == 'HR':
            relevanceAtR = 2
        gain = (2 ** relevanceAtR) - 1
        discount = math.log2(1 + r)
        dcg += gain/discount
    return dcg


averagePrecisionsForMapP = []
averagePrecisionsForMapE = []

for i, rankingPair in enumerate(pairsOfRankingsOf5):
    P = rankingPair[0]
    E = rankingPair[1]
    
    # show the pair
    print ('\nP: ', P, '\nE: ', E, '\n')

    # implement 1 of (binary):
    #   precision at rank k
    #   recall at rank k
    #   average precision   <--
    totalCounter = Counter(P) + Counter(E)
    relevantDocumentCount = totalCounter['R'] + totalCounter['HR']
    
    if relevantDocumentCount == 0:
        # result is irrelevant
        continue
        
    averagePrecisionP = getAveragePrecision(P, relevantDocumentCount)
    averagePrecisionE = getAveragePrecision(E, relevantDocumentCount)
    
    # save for calculating MAP later
    averagePrecisionsForMapP.append(averagePrecisionP)
    averagePrecisionsForMapE.append(averagePrecisionE)

    # implement 2 of (multi-graded):
    #   nDCG at rank k
    #   ERR
    
    # Normalized Discounted Cumulative Gain
    # first we have to determine the perfect ranking. Assuming the P and E results are always
    # different, the perfect ranking would include the results from both lists.
    k = 3
    mergedRanking = P + E
    perfectRanking = sorted(mergedRanking, key=lambda relevance: relevances[relevance])
    print(perfectRanking)
    perfectDcgScoreP = getDiscountedCumulativeGain(mergedRanking[:k])
    
    # only try a few for now
    if i >= 10:
        break;
        
        
        
meanAveragePrecisionP = sum(averagePrecisionsForMapP)/len(averagePrecisionsForMapP)
meanAveragePrecisionE = sum(averagePrecisionsForMapE)/len(averagePrecisionsForMapE)
print('MAP (P) after {} results: {}'.format(len(averagePrecisionsForMapP), meanAveragePrecisionP))
print('MAP (E) after {} results: {}'.format(len(averagePrecisionsForMapE), meanAveragePrecisionE))

59049 pairs of rankings

P:  ('N', 'HR', 'HR', 'HR', 'N') 
E:  ('R', 'R', 'R', 'HR', 'HR') 

['N', 'N', 'R', 'R', 'R', 'HR', 'HR', 'HR', 'HR', 'HR']

P:  ('HR', 'R', 'N', 'N', 'N') 
E:  ('N', 'N', 'HR', 'N', 'HR') 

['N', 'N', 'N', 'N', 'N', 'N', 'R', 'HR', 'HR', 'HR']

P:  ('HR', 'R', 'R', 'HR', 'HR') 
E:  ('N', 'N', 'R', 'HR', 'R') 

['N', 'N', 'R', 'R', 'R', 'R', 'HR', 'HR', 'HR', 'HR']

P:  ('N', 'N', 'N', 'HR', 'R') 
E:  ('HR', 'R', 'HR', 'R', 'HR') 

['N', 'N', 'N', 'R', 'R', 'R', 'HR', 'HR', 'HR', 'HR']

P:  ('HR', 'N', 'R', 'R', 'R') 
E:  ('HR', 'HR', 'HR', 'R', 'N') 

['N', 'N', 'R', 'R', 'R', 'R', 'HR', 'HR', 'HR', 'HR']

P:  ('R', 'R', 'HR', 'N', 'R') 
E:  ('R', 'N', 'N', 'N', 'R') 

['N', 'N', 'N', 'N', 'R', 'R', 'R', 'R', 'R', 'HR']

P:  ('N', 'HR', 'R', 'HR', 'HR') 
E:  ('HR', 'N', 'HR', 'N', 'R') 

['N', 'N', 'N', 'R', 'R', 'HR', 'HR', 'HR', 'HR', 'HR']

P:  ('HR', 'R', 'HR', 'R', 'N') 
E:  ('R', 'N', 'HR', 'HR', 'R') 

['N', 'N', 'R', 'R', 'R', 'R', 'HR', 'HR', 'HR', 'H